In [2]:
# Naive Bayesian / Jongwon Lee
# 2017. 07. 21. -
# Use five pollutant data exposure
# , highest Temperature, highest Percipitation, and highest Air Pressure.
%run  "../../PyUtils/DataFrameUtils.py"
import pandas;

PREV_PEF_POLL = 0;
CLIMATE_POLL = 1;
PREV_PEF_POLL_MAX = 2;

ISCLASSICAL = 1;
PATIENT = 'hand'; # 185, 296
classic_bound = pandas.DataFrame(columns=('RY_bound','YG_bound'));
classic_bound.loc['hand'] = [185, 296];
classic_bound.loc['kuky'] = [295, 472];

print(classic_bound)

# PEF R0 <= 30, 30 < R1 <= 60, R2 > 60
# SO2 -> 0~0.02, 0.021~0.05, 0.051~0.15, 0.151~
# CO -> 0~2, 2.01~9, 9.01~15, 15.01~
# O3 -> 0~0.03, 0.031~0.09, 0.091~0.15, 0.151~
# NO2 -> 0~0.03, 0.031~0.06, 0.061~0.2, 0.201~
# PM10 -> 0~30, 31~80, 81~150, 151~
# SO2 0.05 | CO 9 | O3 0.09 | NO2 0.06 | PM10 80 - (*24) Bad boundary 
# SO2 0.15 | CO 15 | O3 0.15 | NO2 0.2 | PM10 150 - (*24) Very Bad boundary

ISVERYBAD = 0;
if ISVERYBAD == 1 :
    so2_b = 0.15;
    co_b = 15;
    o3_b = 0.15;
    no2_b = 0.2;
    pm10_b = 150;
else :
    so2_b = 0.05;
    co_b = 9;
    o3_b = 0.09;
    no2_b = 0.06;
    pm10_b = 80;

# Tmax, Amax, Pmax boundaries..? -> half and half
tmax_b = 23.40;
amax_b = 1013.30;
pmax_b = 97;


X_VARI_MODE = 0;

if X_VARI_MODE == PREV_PEF_POLL :
    X_variables = ['-3regi','-2regi','-1regi','so2','co','o3','no2','pm10'];
    bound_env = [0, 0, 0, so2_b*24, co_b*24, o3_b*24, no2_b*24, pm10_b*24];
elif X_VARI_MODE == CLIMATE_POLL :
    X_variables = ['tmax','amax','pmax','so2','co','o3','no2','pm10'];
    bound_env = [tmax_b, amax_b, pmax_b, so2_b*24, co_b*24, o3_b*24, no2_b*24, pm10_b*24];
elif X_VARI_MODE == PREV_PEF_POLL_MAX :
    X_variables = ['-3regi','-2regi','-1regi','so2max','comax','o3max','no2max','pm10max'];
    bound_env = [0, 0, 0, so2_b, co_b, o3_b, no2_b, pm10_b];


# bound_env = [0, 0, 0, 0.11, 13.10, 0.57, 0.61, 1150];
# 문제점 2가지
# t_max가 PEF 잴 때의 온도?
# 계속 나쁨으로 유지 ㄴㄴ..
print(bound_env);

# 1. 파일을 읽고
# 3. 베이지안을 써서 확률값을 구하고
# 4. 가장 높은 확률 값을 선택하고
# 5. 정확도를 구한다. 
# 0.196 19.900 0.077 1.176 1072.000  3.700 1023.900  96.000

      RY_bound  YG_bound
hand     185.0     296.0
kuky     295.0     472.0
[0, 0, 0, 1.2000000000000002, 216, 2.16, 1.44, 1920]


In [3]:
# Load AM data and PM data seperately.

data_AM = "http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20"+PATIENT+"%20WHERE%20ampm=\'am\'%20ORDER%20BY%20dateofmeasure";
data_PM = "http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20"+PATIENT+"%20WHERE%20ampm=\'pm\'%20ORDER%20BY%20dateofmeasure";

In [4]:
# Using pandas dataframe structure, store the data.
dfOriginal_AM = LoadDataSet(data_AM, checkForDateTime = False);
dfOriginal_PM = LoadDataSet(data_PM, checkForDateTime = False);
displayDFs (dfOriginal_AM, maxrows = 3);
#displayDFs (dfOriginal_PM, maxrows = 3);
#print(dfOriginal_AM);

NameError: name 'LoadDataSet' is not defined

In [715]:
#print (dfOriginal_AM.count());
#print (dfOriginal_PM.count());
df_AM = dfOriginal_AM;
df_PM = dfOriginal_PM;

In [716]:
#df_AM = df_AM[['dateofmeasure','pefmax','so2','co','o3','no2','pm10','tmax','amax','pmax']];
#df_PM = df_PM[['dateofmeasure','pefmax','so2','co','o3','no2','pm10','tmax','amax','pmax']];
#displayDFs (df_AM, maxrows = 3);
#displayDFs (df_PM, maxrows = 3);
#print(list(df_AM)[0])

if ISCLASSICAL == 1 :
    df_AM['-3regi'] = np.zeros(len(df_AM));
    df_AM['-2regi'] = np.zeros(len(df_AM));
    df_AM['-1regi'] = np.zeros(len(df_AM));
    df_AM['regi'] = np.zeros(len(df_AM));
elif ISCLASSICAL == 1 :
    df_AM['-3regi'] = str(np.zeros(len(df_AM)));
    df_AM['-2regi'] = str(np.zeros(len(df_AM)));
    df_AM['-1regi'] = str(np.zeros(len(df_AM)));
    df_AM['regi'] = str(np.zeros(len(df_AM)));

In [717]:
import numpy as np;
df_AM = df_AM[np.isfinite(df_AM['pefmax'])];
for i in X_variables :
    df_AM = df_AM[np.isfinite(df_AM[i])];
if 'pmax' in X_variables :
    df_AM = df_AM[np.not_equal(df_AM['pmax'], -1000.)];  # Percipitation data 
    
#displayDFs (df_AM);
print(df_AM);

    cname ampm timeofday dateofmeasure    pef1    pef2    pef3  pefmax indexpef   so2     co    o3   no2     pm10  temperaturec  rainfallmm  windspeedms  precipitationpercent  vaporpressurehpa  dewpointtemperaturec  airpressurehpa  sealevelpressurehpa  groundtemperaturec  winddirection   tmax     amax    pmax   tmin     amin   pmin  tmode    amode   pmode  tmaxlesstmin  amaxlessamin  pmaxlesspmin  so2max  so2maxtime  so2min  so2mintime  comax  comaxtime  comin  comintime  o3max  o3maxtime  o3min  o3mintime  no2max  no2maxtime  no2min  no2mintime  pm10max  pm10maxtime  pm10min  pm10mintime  -3regi  -2regi  -1regi  regi
0    hand   am  07:40:00    2014-12-24 319.000 305.000 319.000 319.000        G 0.196 19.900 0.077 1.176 1072.000         1.900         nan        0.300                83.000             5.800                -0.600        1017.300             1021.600              -0.400          0.000  3.700 1023.900  96.000 -6.600 1000.000 56.000  1.900 1023.900  65.000        10.300   

In [718]:
df_AM = df_AM.reset_index(drop=True);
#print(df_AM);

In [719]:
#print(df_AM)
if ISCLASSICAL == 0 :
    bound_1 = df_AM.pefmax.quantile(.3);
    bound_2 = df_AM.pefmax.quantile(.6);
elif ISCLASSICAL == 1 :
    bound_1 = classic_bound.loc[PATIENT]['RY_bound'];
    bound_2 = classic_bound.loc[PATIENT]['YG_bound'];
    
for i in range(0, len(df_AM)):
    if df_AM.loc[i]['pefmax'] < bound_1 :
        df_AM.set_value(i, 'regi', 0);
    elif df_AM.loc[i]['pefmax'] < bound_2 :
        df_AM.set_value(i, 'regi', 1);
    else :
        df_AM.set_value(i, 'regi', 2);

for i in range(3, len(df_AM)) :
    for j in range(3, 0, -1) :
        if df_AM.loc[i-j]['pefmax'] < bound_1 :
            df_AM.set_value(i, '-'+str(j)+'regi', 0);
        elif df_AM.loc[i-j]['pefmax'] < bound_2 :
            df_AM.set_value(i, '-'+str(j)+'regi', 1);
        else :
            df_AM.set_value(i, '-'+str(j)+'regi', 2);
        
        
print(bound_1, bound_2);
#print(df_AM);
#print('-'+str(j)+'regi');

185.0 296.0


In [720]:
df_AM = df_AM[X_variables+['regi']];
df_AM = df_AM[3:len(df_AM)];
df_AM = df_AM.reset_index(drop=True);
print(df_AM);

     -3regi  -2regi  -1regi   so2     co    o3   no2     pm10  regi
0     2.000   2.000   2.000 0.136 17.000 0.173 0.914  867.000 2.000
1     2.000   2.000   2.000 0.170 25.700 0.070 1.394 1359.000 2.000
2     2.000   2.000   2.000 0.219 23.200 0.062 1.404 1251.000 2.000
3     2.000   2.000   2.000 0.269 24.800 0.216 1.217 3064.000 1.000
..      ...     ...     ...   ...    ...   ...   ...      ...   ...
595   1.000   1.000   1.000 0.065  9.500 0.527 0.449  420.000 1.000
596   1.000   1.000   1.000 0.065 10.900 0.440 0.577  691.000 1.000
597   1.000   1.000   1.000 0.067 10.500 0.632 0.407  804.000 1.000
598   1.000   1.000   1.000 0.072 11.400 0.390 0.548  877.000 1.000

[599 rows x 9 columns]


In [723]:
sample_num = 100;
avg_accuracy = 0.;
df_Prob_avg = pandas.DataFrame(columns=('R0','R1','R2'));
df_Prob_min = pandas.DataFrame(columns=('R0','R1','R2'));
df_Prob_max = pandas.DataFrame(columns=('R0','R1','R2'));

# Iterate sampling and calculate average of accuracy
for sample_count in range(0, sample_num):
    # Sampling
    msk = np.random.rand(len(df_AM)) <= (2./3.)
    df_AM_train = df_AM[msk];
    df_AM_test = df_AM[~msk];
    #df_AM_train = df_AM_train.drop('timeofday', axis = 1);
    #df_AM_train = df_AM_train.drop('dateofmeasure', axis = 1);
    df_AM_train = df_AM_train.reset_index(drop=True);
    df_AM_test = df_AM_test.reset_index(drop=True);
    #print(df_AM_train);
    #print(df_AM_test);

    df_Result = pandas.DataFrame(columns=('R0','R1','R2','Sum'));
    df_Prob = pandas.DataFrame(columns=('R0','R1','R2'));
    df_Correct = pandas.DataFrame(np.zeros((len(df_AM_test), 1)), columns = ['iscorrect']);

    # Counting
    for i in range(0, len(df_AM_train)) :
        index_str = '';

        for j in range(0,3) :
            if X_VARI_MODE == PREV_PEF_POLL or X_VARI_MODE == PREV_PEF_POLL_MAX :
                index_str = index_str + str(int(df_AM_train.loc[i][j]));
            elif X_VARI_MODE == CLIMATE_POLL :
                if df_AM_train.loc[i][j] <= bound_env[j] :
                    index_str = index_str + '0';
                else :
                    index_str = index_str + '1';   

        for j in range(3,8) :
            if df_AM_train.loc[i][j] <= bound_env[j] :
                index_str = index_str + '0';
            else :
                index_str = index_str + '1';

        if index_str not in df_Result.index :
            df_Result.loc[index_str] = [0, 0, 0, 0];
            df_Prob.loc[index_str] = [0., 0., 0.];
            
        if index_str not in df_Prob_avg.index :
            df_Prob_avg.loc[index_str] = [0, 0, 0];
            df_Prob_min.loc[index_str] = [1, 1, 1];
            df_Prob_max.loc[index_str] = [0, 0, 0];

        df_Result.set_value(index_str, 'Sum', df_Result.loc[index_str]['Sum']+1);
        if df_AM_train.loc[i]['regi'] == 0 :
            df_Result.set_value(index_str, 'R0', df_Result.loc[index_str]['R0']+1);
        elif df_AM_train.loc[i]['regi'] == 1 :
            df_Result.set_value(index_str, 'R1', df_Result.loc[index_str]['R1']+1);
        else :
            df_Result.set_value(index_str, 'R2', df_Result.loc[index_str]['R2']+1);

    # Calculate Prob
    for i in df_Result.index :
        df_Prob.set_value(i, 'R0', df_Result.loc[i]['R0']/df_Result.loc[i]['Sum']);
        df_Prob.set_value(i, 'R1', df_Result.loc[i]['R1']/df_Result.loc[i]['Sum']);
        df_Prob.set_value(i, 'R2', df_Result.loc[i]['R2']/df_Result.loc[i]['Sum']);
        
    # Store Prob
    for i in df_Result.index :
        for j in ['R0', 'R1', 'R2'] :
            if df_Prob_min.loc[i][j] > df_Prob.loc[i][j] :
                df_Prob_min.loc[i][j] = df_Prob.loc[i][j];
            if df_Prob_max.loc[i][j] < df_Prob.loc[i][j] :
                df_Prob_max.loc[i][j] = df_Prob.loc[i][j];
            df_Prob_avg.loc[i][j] = df_Prob_avg.loc[i][j] + df_Prob.loc[i][j];

    # Calculate Accuracy
    for i in range(0, len(df_AM_test)) :
        index_str = '';

        for j in range(0,3) :
            if X_VARI_MODE == PREV_PEF_POLL or X_VARI_MODE == PREV_PEF_POLL_MAX :
                index_str = index_str + str(int(df_AM_test.loc[i][j]));
            elif X_VARI_MODE == CLIMATE_POLL :
                if df_AM_test.loc[i][j] <= bound_env[j] :
                    index_str = index_str + '0';
                else :
                    index_str = index_str + '1'; 

        for j in range(3,8) :
            if df_AM_test.loc[i][j] <= bound_env[j] :
                index_str = index_str + '0';
            else :
                index_str = index_str + '1';

        if index_str not in df_Result.index :
            df_Correct.set_value(i, 'iscorrect' , NaN);
        else :    
            maxN = 0;
            maxRegi = -1;

            for j in range(0,3) :
                if maxN < df_Result.loc[index_str][j] :
                    maxN = df_Result.loc[index_str][j];
                    maxRegi = j;

            if df_AM_test.loc[i]['regi'] == maxRegi :
                df_Correct.set_value(i, 'iscorrect' , 1);

    avg_accuracy = avg_accuracy + sum(df_Correct)/len(df_Correct);
    
avg_accuracy = avg_accuracy / (sample_count + 1);
df_Prob_avg = df_Prob_avg / (sample_count + 1);
print('Sampling number: ', sample_count + 1);
print(avg_accuracy);
print(df_Prob_min);
print(df_Prob_avg);
print(df_Prob_max); 
#print(sum(df_Correct)/len(df_Correct));
#print(len(df_Correct) - df_Correct.count());
#print(df_Correct.to_string());

        
#print (index, 178 * 9)
#print(df_AM_train)
#print(df_Result)
#print(df_Result.to_string())
#print(df_Prob.to_string())
# print(df_Result[[0,1,2]].max(axis=1))
#print(sum(df_Result[['Sum']]));

Sampling number:  100
iscorrect   0.784
dtype: float64
            R0    R1    R2
22200000 0.000 0.375 0.222
22100001 0.000 1.000 0.000
21100000 0.000 0.625 0.107
11100000 0.000 0.872 0.060
...        ...   ...   ...
22100011 0.000 0.000 1.000
21200001 0.000 1.000 0.000
11100011 0.000 1.000 0.000
22200001 0.000 1.000 0.000

[23 rows x 3 columns]
            R0    R1    R2
22200000 0.000 0.540 0.460
22100001 0.000 0.890 0.000
21100000 0.000 0.758 0.242
11100000 0.000 0.904 0.096
...        ...   ...   ...
22100011 0.000 0.000 0.640
21200001 0.000 0.870 0.000
11100011 0.000 0.850 0.000
22200001 0.000 0.650 0.000

[23 rows x 3 columns]
            R0    R1    R2
22200000 0.000 0.778 0.625
22100001 0.000 1.000 0.000
21100000 0.000 0.893 0.375
11100000 0.000 0.940 0.128
...        ...   ...   ...
22100011 0.000 0.000 1.000
21200001 0.000 1.000 0.000
11100011 0.000 1.000 0.000
22200001 0.000 1.000 0.000

[23 rows x 3 columns]
